In [1]:
#Importing the needed packages
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline 

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB


import random
import time
import os
import math

# Null Handling
gross_bookings_usd: 97% <br/>
srch_query_affinity_score: 94%  <br/>
orig_destination_distance: 32% #juul fixt deze <br/>
prop_location_score2: 22% #replaced by mean <br/>
prop_location_score1: 0% <br/>
visitor_hist_adr_usd: 95% <br/>
visitor_hist_starrating: 95% full corr. <br/>


In [2]:
#loading the data
expedia_df = pd.read_csv('training_set_VU_DM_2014.csv')
test_df = pd.read_csv('test_set_VU_DM_2014.csv')

In [20]:
expedia_df[['srch_id','price_usd','prop_log_historical_price']].head(20)

,srch_id,price_usd,prop_log_historical_price
0,1,104.77,4.95
1,1,170.74,5.03
2,1,179.80,4.92
3,1,602.77,4.39
4,1,143.58,4.93
5,1,195.32,5.20
6,1,129.35,4.81
7,1,85.37,4.14
8,1,150.05,5.18
9,1,280.69,5.15


In [10]:
def bin_cont(data, variable_name, nbins):
    labels_ = [str(x) for x in np.arange(nbins)]
    # assign categorical labels to n bins
    data[variable_name] = pd.cut(data[variable_name],nbins, labels=labels_)
    # Transform categorical variable to several binary dummy variables
    data = pd.get_dummies(data,columns=[variable_name])
    return data
    

def normalize(data, variable):
    d = data[variable]
    mean = np.mean(d)
    norm_d = [(x-mean)/mean for x in d]
    data[variable] = norm_d
    return data

def pp_time(data):
    data["date_time"] = pd.to_datetime(data["date_time"])
    data["year"] = data["date_time"].dt.year
    data["month"] = data["date_time"].dt.month
    return data

def join_comps(data):
    comp_vars = ['comp1_rate','comp1_inv','comp1_rate_percent_diff',
                'comp2_rate','comp2_inv','comp2_rate_percent_diff',
                'comp3_rate','comp3_inv','comp3_rate_percent_diff',
                'comp4_rate','comp4_inv','comp4_rate_percent_diff',
                'comp5_rate','comp5_inv','comp5_rate_percent_diff',
                'comp6_rate','comp6_inv','comp6_rate_percent_diff',
                'comp7_rate','comp7_inv','comp7_rate_percent_diff',
                'comp8_rate','comp8_inv','comp8_rate_percent_diff']
    data = remove_comp_outliers(data, *[x for x in comp_vars if 'percent' in x])
    data = combine_comps(data,comp_vars)
    data.drop(comp_vars,axis=1)
    
    def remove_comp_outliers(data, *variables):
        # Removes outliers from the percent_diff set. Only high-end outliers are removed
        for var in variables:
            median = data[var].median()
            quantile = data[var].quantile(0.9)
            thresh = median + (1.5*(quantile-median))
            removed_outliers = []
            for i,point in enumerate(data[var].values):
                if not math.isnan(point):
                    if point > thresh:
                        removed_outliers.append(None)
                    else:
                        removed_outliers.append(point)
                else:
                    removed_outliers.append(None)
            data[var] = removed_outliers
        return data
    def combine_comps(data, comp_vars):
        def combine_rate_or_inv(row, is_inv):
            if is_inv:
                print('inv!',row)
            non_null = [x for x in row if not math.isnan(x)]
            if len(non_null) != 0:
                return sum(non_null)/len(non_null)
            else:
                return None

        def combine_percent_diff(percent, signs):
            rel_dif = np.array(percent) * np.array(signs)
            rel_diff = [x for x in rel_dif if not math.isnan(x)]
            if len(rel_diff) != 0:
                return sum(rel_diff)/len(rel_diff)
            else:
                return None

        rate_vars = [x for x in comp_vars if 'rate' in x and 'percent' not in x]
        inv_vars  = [x for x in comp_vars if 'inv'  in x]
        percent_vars = [x for x in comp_vars if 'percent' in x]
        print(rate_vars)
        print(inv_vars)
        print(percent_vars)
        comp_rate = []
        comp_inv = []
        comp_diff = []
        for i,row in enumerate(data[rate_vars].values):
            comp_rate.append(combine_rate_or_inv(row,False))
            comp_inv.append(combine_rate_or_inv(data[inv_vars].values[i],True))

        for i,row in enumerate(data[percent_vars].values):
            signs = data[rate_vars].values[i]
            comp_diff.append(combine_percent_diff(row, signs))
        data['comp_rate'] = comp_rate
        data['comp_inv'] = comp_inv
        data['comp_diff'] = comp_diff
        print('Done')
        return data
    return data
    
def normalize_mult(data, *variables):
    for var in variables:
        data = normalize(data, var)
    return data


In [ ]:
def preprocess(data):
    data = pp_time(data) # drops date time adds year, month
    data = join_comps(data) # drops compx vars adds joined comp vars (and outliers)
    data = normalize_mult(data, 'prop_rev')
    data[var_name]= change_null(data,var_name) #fills nullvalues in variable with the mean 
    